<a href="https://colab.research.google.com/github/sodonne6/Data_Augmentation_for_AVSR/blob/main/Av_Hubert_first_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
%pip -q install -U condacolab
import condacolab
condacolab.install() #restarts runtime

✨🍰✨ Everything looks OK!


In [13]:
#create and activate python 3.8 env
%%bash
eval "$(conda shell.bash hook)"
conda env remove -n av_hubert || true # Remove the environment if it exists
conda create -n av_hubert python=3.8
conda activate av_hubert

# clone and install dependencies inside env
git clone https://github.com/facebookresearch/av_hubert.git /content/av_hubert
cd /content/av_hubert
git submodule init
git submodule update

#show content for sanity check
pwd
ls -lah


Remove all packages in environment /usr/local/envs/av_hubert:


## Package Plan ##

  environment location: /usr/local/envs/av_hubert


The following packages will be REMOVED:

  _libgcc_mutex-0.1-conda_forge
  _openmp_mutex-4.5-2_gnu
  bzip2-1.0.8-hda65f42_8
  ca-certificates-2025.8.3-hbd8a1cb_0
  ld_impl_linux-64-2.44-h1423503_1
  libffi-3.4.6-h2dba641_1
  libgcc-15.1.0-h767d61c_5
  libgcc-ng-15.1.0-h69a702a_5
  libgomp-15.1.0-h767d61c_5
  liblzma-5.8.1-hb9d3cd8_2
  liblzma-devel-5.8.1-hb9d3cd8_2
  libnsl-2.0.1-hb9d3cd8_1
  libsqlite-3.50.4-h0c1763c_0
  libuuid-2.41.2-he9a06e4_0
  libxcrypt-4.4.36-hd590300_1
  libzlib-1.3.1-hb9d3cd8_2
  ncurses-6.5-h2d0b736_3
  openssl-3.5.3-h26f9b46_1
  pip-24.3.1-pyh8b19718_0
  python-3.8.20-h4a871b0_2_cpython
  readline-8.2-h8c095d6_2
  setuptools-75.3.0-pyhd8ed1ab_0
  tk-8.6.13-noxft_hd72426e_102
  wheel-0.45.1-pyhd8ed1ab_0
  xz-5.8.1-hbcc6ac9_2
  xz-gpl-tools-5.8.1-hbcc6ac9_2
  xz-tools-5.8.1-hb9d3cd8_2



Preparing transaction: - done
Verify



==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c conda-forge conda


fatal: destination path '/content/av_hubert' already exists and is not an empty directory.


In [16]:
# install deps inside the conda env
%%bash
eval "$(conda shell.bash hook)"
conda activate av_hubert
cd /content/av_hubert/

# 0) Repo requirements
pip install -r requirements.txt

# 1) Pin NumPy so np.float exists
pip install --no-cache-dir "numpy==1.23.5"

# 2) Torch (GPU build via pip cu117 wheels) — replace any conda torch
conda remove -y pytorch torchvision torchaudio pytorch-cuda || true
pip uninstall -y torch torchvision torchaudio || true
pip install --no-cache-dir \
  torch==1.13.1+cu117 \
  torchvision==0.14.1+cu117 \
  torchaudio==0.13.1 \
  --extra-index-url https://download.pytorch.org/whl/cu117

# 3) Install fairseq from the bundled submodule (editable) WITHOUT pulling deps
cd fairseq
pip install -e . --no-deps
cd ..

# 4) Force the config stack we need (provides II/MISSING)
pip install --no-cache-dir omegaconf==2.1.1 hydra-core==1.1.0

# 5) Tiny import check
python - << 'PY'
import torch, fairseq, numpy as np
from omegaconf import II, MISSING
import hydra
print("Torch:", torch.__version__, "| CUDA available:", torch.cuda.is_available())
print("Fairseq import OK")
print("NumPy:", np.__version__)
print("OmegaConf II/MISSING present:", II is not None and MISSING is not None)
print("Hydra:", hydra.__version__)
PY


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 304.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 433.6 MB/s eta 0:00:00
Obtaining file:///content/av_hubert/fairseq
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for fairs


PackagesNotFoundError: The following packages are missing from the target environment:
  - pytorch
  - torchaudio
  - pytorch-cuda
  - torchvision


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fairseq 1.0.0a0+afc77bd requires hydra-core<1.1, but you have hydra-core 1.1.0 which is incompatible.
fairseq 1.0.0a0+afc77bd requires omegaconf<2.1, but you have omegaconf 2.1.1 which is incompatible.


In [4]:
##different version of omega and hydra -> fixed apprently
#%%bash
#eval "$(conda shell.bash hook)"
#conda activate av_hubert
#cd /content/av_hubert
#
## Upgrade config stack to match fairseq's expectations (provides `II`, `MISSING`)
#pip uninstall -y omegaconf hydra-core
#pip install --no-cache-dir omegaconf==2.1.1 hydra-core==1.1.0
#
## Sanity import
#python - << 'PY'
#from omegaconf import II, MISSING
#print("OmegaConf OK:", II is not None, MISSING is not None)
#import hydra
#print("Hydra OK, version:", hydra.__version__)
#PY


Found existing installation: omegaconf 1.4.1
Uninstalling omegaconf-1.4.1:
  Successfully uninstalled omegaconf-1.4.1
Found existing installation: hydra-core 0.11.3
Uninstalling hydra-core-0.11.3:
  Successfully uninstalled hydra-core-0.11.3
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141213 sha256=972221a3223dbe76bce5a33d0ae7d88cb3db28eadbb86c99775a1240f6b681c6
  Stored in directory: /tmp/pip-ephem-wheel-cache-9nlfip9f/wheels/c8/d0/ab/d43c02eaddc5b9004db86950802442ad9a26f279c619e28da0
Successfully built antlr4-python3-runtime
OmegaConf OK: True True
Hydra OK, version: 1.1.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fairseq 1.0.0a0+afc77bd requires hydra-core<1.1, but you have hydra-core 1.1.0 which is incompatible.
fairseq 1.0.0a0+afc77bd requires omegaconf<2.1, but you have omegaconf 2.1.1 which is incompatible.


In [5]:
##fix numpy
#%%bash
#eval "$(conda shell.bash hook)"
#conda activate av_hubert
## Pin numpy to a pre-1.24 version so np.float exists
#pip install --no-cache-dir "numpy==1.23.5"
#
## Quick check
#python - << 'PY'
#import numpy as np
#print("NumPy:", np.__version__)
## should not raise
#_ = np.float
#print("np.float available:", True)
#PY


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 160.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
NumPy: 1.23.5
np.float available: True


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fairseq 1.0.0a0+afc77bd requires hydra-core<1.1, but you have hydra-core 1.1.0 which is incompatible.
fairseq 1.0.0a0+afc77bd requires omegaconf<2.1, but you have omegaconf 2.1.1 which is incompatible.
<stdin>:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [6]:
#CODE BEGINNING -> MOUNT GOOGLE DRIVE
#NEEDS TO BE DONE EVERYTIME

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
%%bash
eval "$(conda shell.bash hook)"
conda activate av_hubert
cd /content  # stay outside the repo root

# Put the avhubert *subfolder* on PYTHONPATH so top-level imports work
PYTHONPATH=/content/av_hubert/avhubert:$PYTHONPATH \
python -u - << 'PY'
import os, torch, importlib

# 1) Import fairseq first
import fairseq
from fairseq.tasks import TASK_REGISTRY
print("fairseq OK; registry size before:", len(TASK_REGISTRY))

# 2) Force-import the files that perform registration
#    (these use @register_task / @register_model decorators)
importlib.import_module("hubert_pretraining")  # registers 'av_hubert_pretraining'
importlib.import_module("hubert")              # registers the AV-HuBERT model

# 3) Verify registration
from fairseq.tasks import TASK_REGISTRY
print("Contains 'av_hubert_pretraining':", "av_hubert_pretraining" in TASK_REGISTRY)

# 4) Load checkpoint
from fairseq.checkpoint_utils import load_model_ensemble_and_task
ckpt_path = "/content/drive/MyDrive/AVSR_Colab/base_lrs3_iter5.pt"  # adjust if needed
assert os.path.exists(ckpt_path), f"Checkpoint not found: {ckpt_path}"

models, cfg, task = load_model_ensemble_and_task([ckpt_path])
model = models[0]

print("✅ Loaded AV-HuBERT checkpoint")
print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
print("Model:", type(model).__name__, "| Task:", type(task).__name__)
PY


fairseq OK; registry size before: 25
Contains 'av_hubert_pretraining': True
2025-09-23 15:47:32 | INFO | hubert_pretraining | current directory is /content
2025-09-23 15:47:32 | INFO | hubert_pretraining | AVHubertPretrainingTask Config {'_name': 'av_hubert_pretraining', 'data': '/checkpoint/bshi/data/lrs3//video/hubert/multimodal-tsv-2/', 'labels': ['km'], 'label_dir': '/checkpoint/bshi/data/lrs3//video/hubert/stitch-iters/lrs3-iter3-feat-d0.5-iter3-l12c2000/', 'label_rate': 25, 'sample_rate': 25, 'normalize': True, 'enable_padding': False, 'max_sample_size': 500, 'min_sample_size': 5, 'max_trim_sample_size': '${task.max_sample_size}', 'single_target': False, 'random_crop': False, 'pad_audio': True, 'pdb': False, 'stack_order_audio': 4, 'skip_verify': False, 'image_aug': True, 'image_crop_size': 88, 'image_mean': 0.421, 'image_std': 0.165, 'modalities': ['audio', 'video'], 'is_s2s': False, 'tokenizer_bpe_name': None, 'tokenizer_bpe_model': None, 'noise_wav': None, 'noise_prob': 0.0, '

In [40]:
input_video_path = "/content/drive/MyDrive/AVSR_Colab/sample_video.mp4"